# Setup:

Select GPUs before starting this ipython server. As an example, to use /gpu:4 and /gpu:6 on the server, use the following command:

*$ export CUDA_VISIBLE_DEVICES=4,6*

In case this variable is not set or mulitple devices are selected, TensorFlow will allocate memory on **all** devices, but will run only on /gpu:0.

In [1]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensortools as tt

import model.lstmconv2d_encoder_decoder as model

In [2]:
# Global config parameters
INPUT_SEQ_LENGTH = 10
OUTPUT_SEQ_LENGTH = 10

FRAME_WIDTH = 64
FRAME_HEIGHT = 64
FRAME_CHANNELS = 1

MOVING_AVERAGE_DECAY = 0.9999
NUM_EPOCHS_PER_DECAY = 75.0 # used if FIXED_NUM_STEPS_PER_DECAY is None
LEARNING_RATE_DECAY_FACTOR = 0.2
INITIAL_LEARNING_RATE = 0.001

FIXED_NUM_STEPS_PER_DECAY = 10000

LAMBDA = 5e-4

BATCH_SIZE = 16  # per GPU!

TRAIN_DIR = 'train_moving_mnist'
MAX_STEPS = 30000

GPU_MEMORY_FRACTION = 1.0
GPU_ALLOW_GROWTH = True
NUM_GPUS = 2
MEMORY_DEVICE = '/cpu:0'

# Input Data

In [3]:
dataset_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(BATCH_SIZE * NUM_GPUS,
                                                                INPUT_SEQ_LENGTH + OUTPUT_SEQ_LENGTH)
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(BATCH_SIZE * NUM_GPUS,
                                                                INPUT_SEQ_LENGTH + OUTPUT_SEQ_LENGTH)
# dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(BATCH_SIZE * NUM_GPUS,
#                                                                INPUT_SEQ_LENGTH + OUTPUT_SEQ_LENGTH)

# For manual verification of used parameters
decay_after_steps = NUM_EPOCHS_PER_DECAY * dataset_train.dataset_size // (BATCH_SIZE*NUM_GPUS)
if FIXED_NUM_STEPS_PER_DECAY is not None:
    decay_after_steps = FIXED_NUM_STEPS_PER_DECAY
print("Learning rate decay every {} steps".format(decay_after_steps))

File mnist.h5 has already been downloaded.
File mnist.h5 has already been downloaded.
Learning rate decay every 10000 steps


# Training

In [4]:
def tower_loss(x, y_):
    """Calculate the total loss on a single tower.
    Args:
        scope: unique prefix string identifying the tower, e.g. 'tower_0'
    Returns:
        Tensor of shape [] containing the total loss for a batch of data
    """
    # Build inference Graph.
    predictions = model.inference(x,
                                  y_,
                                  FRAME_CHANNELS, 
                                  INPUT_SEQ_LENGTH, 
                                  LAMBDA)

    # Build the portion of the Graph calculating the losses. Note that we will
    # assemble the total_loss using a custom function below.
    total_loss, loss = model.loss(predictions, y_)

    # Compute the moving average of all individual losses and the total loss.
    # Generate moving averages of all losses and associated summaries
    with tf.device('/cpu:0'):
        loss_averages_op = tt.board.loss_summary([total_loss, loss] +
                                                 tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

    with tf.control_dependencies([loss_averages_op]):
        total_loss = tf.identity(total_loss)
    
    return total_loss

In [5]:
def train(global_step):
    """Train sequence model.
    Create an optimizer and apply to all trainable variables. Add moving
    average for all trainable variables.
    Args:
        total_cost: Total loss from loss function including regularization terms.
        cost: Raw loss from loss function without regularization terms.
        global_step: Integer Variable counting the number of training steps
                     processed.
    Returns:
        train_op: op for training.
    """
    # Variables that affect learning rate
    num_batches_per_epoch = dataset_train.dataset_size / (BATCH_SIZE * NUM_GPUS)
    decay_steps = num_batches_per_epoch * NUM_EPOCHS_PER_DECAY
    
    if FIXED_NUM_STEPS_PER_DECAY is not None:
        decay_steps = FIXED_NUM_STEPS_PER_DECAY

    # Decay the learning rate exponentially based on the number of steps
    lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                    global_step,
                                    decay_steps,
                                    LEARNING_RATE_DECAY_FACTOR,
                                    staircase=True)
    tf.scalar_summary('learning_rate', lr)

    # Compute gradients
    opt = tf.train.AdamOptimizer(lr)
    # Calculate the gradients for each model tower.
    tower_grads = []
    tower_losses = []
    for i in xrange(NUM_GPUS):
        with tf.device('/gpu:%d' % i, ):
            with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                # Calculate the loss for one tower of the model. This function constructs 
                # the entire model but shares the variables across all towers.
                t_loss = tower_loss(x[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :, :, :, :],
                                    y_[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :, :, :, :])
                tower_losses.append(t_loss)

                # Reuse variables for the next tower.
                tf.get_variable_scope().reuse_variables()

                # Calculate the gradients for the batch of data on this tower.
                grads = opt.compute_gradients(t_loss)

                # Keep track of the gradients ackeep_probross all towers.
                tower_grads.append(grads)

    # We must calculate the mean of each gradient.
    # This is also the synchronization point across all towers.
    grads = tt.training.average_gradients(tower_grads)
    
    total_loss = tf.reduce_mean(tower_losses)

    # Apply the gradients to adjust the shared variables.
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

    tt.board.variables_histogram_summary()

    # Add histograms for gradients
    tt.board.gradients_histogram_summary(grads)

    # Track the moving averages of all trainable variables
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    # Group all updates to into a single train op.
    train_op = tf.group(apply_gradient_op, variables_averages_op, name="train_op")

    return train_op, total_loss

# TensorFlow Session (Main)

In [6]:
with tf.Graph().as_default(), tf.device('/cpu:0'):
    global_step = tf.Variable(0, trainable=False)
    
    x = tf.placeholder(tf.float32, [None, INPUT_SEQ_LENGTH, FRAME_HEIGHT, FRAME_WIDTH, FRAME_CHANNELS], "X")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_SEQ_LENGTH, FRAME_HEIGHT, FRAME_WIDTH, FRAME_CHANNELS], "Y_")
    
    # train the model
    train_op, total_loss = train(global_step)

    # Create a saver and merge all summaries
    saver = tf.train.Saver(tf.all_variables())
    summary_op = tf.merge_all_summaries()
    
    # Create the graph, etc.
    init_op = tf.initialize_all_variables()
    
    # Create a session for running operations in the Graph
    gpu_options = tf.GPUOptions(
        per_process_gpu_memory_fraction=GPU_MEMORY_FRACTION,
        allow_growth=GPU_ALLOW_GROWTH)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        
        # Initialize the variables (like the epoch counter)
        sess.run(init_op)
        
        # Visualize graph
        tt.visualization.show_graph(sess.graph_def)
        
        # Start input enqueue threads
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        summary_writer = tf.train.SummaryWriter(TRAIN_DIR, sess.graph)
        
        dataset_train.reset()

        try:
            step = 0
            while not coord.should_stop():
                step += 1
                
                if (step > MAX_STEPS):
                    break

                start_time = time.time()
                
                batch = dataset_train.get_batch()      
                batch_x = batch[:,0:INPUT_SEQ_LENGTH,:,:,:]
                batch_y = batch[:,INPUT_SEQ_LENGTH:INPUT_SEQ_LENGTH+OUTPUT_SEQ_LENGTH,:,:,:]

                _, loss_value = sess.run([train_op, total_loss],
                                         feed_dict={x: batch_x, y_: batch_y})
                duration = time.time() - start_time

                assert not np.isnan(loss_value), 'Model diverged with cost = NaN'

                if step % 10 == 0:
                    num_examples_per_step = BATCH_SIZE * NUM_GPUS
                    examples_per_sec = num_examples_per_step / duration
                    sec_per_batch = float(duration)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print (format_str % (datetime.now().time(), step, loss_value,
                                         examples_per_sec, sec_per_batch))

                if step % 100 == 0:
                    summary_str = sess.run(summary_op, feed_dict={x: batch_x, y_: batch_y})
                    summary_writer.add_summary(summary_str, step)
                    summary_writer.flush() 

                # Save the model checkpoint periodically.
                if step % 1000 == 0 or (step + 1) == MAX_STEPS:
                    checkpoint_path = os.path.join(TRAIN_DIR, 'model.ckpt')
                    saver.save(sess, checkpoint_path, global_step=step)

        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            # When done, ask the threads to stop
            coord.request_stop()

        # Wait for threads to finish
        coord.join(threads)

17:47:08.491424: step 10, loss = 127.48 (26.5 examples/sec; 0.603 sec/batch)
17:47:14.526157: step 20, loss = 104.02 (26.6 examples/sec; 0.603 sec/batch)
17:47:20.578274: step 30, loss = 86.83 (26.5 examples/sec; 0.604 sec/batch)
17:47:26.619475: step 40, loss = 84.17 (26.4 examples/sec; 0.606 sec/batch)
17:47:32.661147: step 50, loss = 79.30 (26.5 examples/sec; 0.604 sec/batch)
17:47:38.700782: step 60, loss = 83.81 (26.5 examples/sec; 0.603 sec/batch)
17:47:44.746399: step 70, loss = 82.29 (26.5 examples/sec; 0.604 sec/batch)
17:47:50.793614: step 80, loss = 83.95 (26.4 examples/sec; 0.607 sec/batch)
17:47:56.831277: step 90, loss = 84.54 (26.5 examples/sec; 0.604 sec/batch)
17:48:02.877851: step 100, loss = 82.68 (26.4 examples/sec; 0.606 sec/batch)
17:48:14.127894: step 110, loss = 83.46 (26.5 examples/sec; 0.605 sec/batch)
17:48:20.166633: step 120, loss = 81.88 (26.5 examples/sec; 0.603 sec/batch)
17:48:26.225224: step 130, loss = 71.67 (26.3 examples/sec; 0.608 sec/batch)
17:48:

KeyboardInterrupt: 